# Fit function

based off jeremy howards great tutorial! https://pytorch.org/tutorials/beginner/nn_tutorial.html

## Setup

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt
import numpy as np

from pathlib import Path
import requests
import pickle
import gzip
import torch

In [3]:
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"
FILENAME = "mnist.pkl.gz"

In [4]:
def unzip_data():
    with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    
    return x_train, y_train, x_valid, y_valid

## PyTorch Datasets

abstract class where you need to implement

__len__ & __getitem__ methods 

In [5]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [6]:
class MnistDataset(Dataset):
    def __init__(self, is_valid=False):
        x_train, y_train, x_valid, y_valid = unzip_data()
        
        self.x = x_train if is_valid else x_valid
        self.y = y_train if is_valid else y_valid
        self.len = self.x.shape[0] 
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.len

In [7]:
train_ds = MnistDataset()
valid_ds = MnistDataset(is_valid=True)

## PyTorch Dataloader

manages getting batches for our training loop from our dataloader

returns a python interator

In [8]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [9]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

def to_device(x, y):
    return x.to(dev), y.to(dev)

def get_data(bs):
    train_ds = MnistDataset()
    valid_ds = MnistDataset(is_valid=True)

    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
    
    train_dl = WrappedDataLoader(train_dl, to_device)
    valid_dl = WrappedDataLoader(valid_dl, to_device)
    
    return train_dl, valid_dl

In [10]:
from torch import nn
import torch.nn.functional as F

loss_func = F.cross_entropy

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

class Mnist_NN(nn.Module):
    def __init__(self, n_h):
        super().__init__()
        self.inp = nn.Linear(784, n_h)
        self.hid = nn.Linear(n_h, n_h)
        self.out = nn.Linear(n_h, 10)
        
    def forward(self, xb):
        ab1 = F.relu(self.inp(xb))
        ab2 = F.relu(self.hid(ab1))
        ab3 = F.relu(self.out(ab2))
        return ab3

In [13]:
from torch import optim

def get_model():
    model = Mnist_NN(100)
    opt = optim.SGD(model.parameters(), lr=0.02)
    return model, opt

## Write loss_batch and fit functions

<a id="q_1" >

In [30]:
to4dec = lambda a: np.around(a, decimals=4) if not a==None else None

def print_epoch_progress(epoch, train_loss=None, valid_loss=None, metrics=[]):
    metrics = [f"{key}: {to4dec(value)}" for key, value in metrics.items()]
    print(epoch, 'train loss: ', to4dec(train_loss),'valid loss: ', to4dec(valid_loss,), ' '.join(metrics))

In [31]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    
    # if validation dont update weights (you cant learn from them)

    return 

In [32]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        
        model.train() # training
        
        model.eval() # validation

        
        print_epoch_progress(epoch, training_loss, validation_loss, {'accuracy': epoch_accuracy})

In [34]:
model, opt = get_model()
train_dl, valid_dl = get_data(64)
fit(10, model, loss_func, opt, train_dl, valid_dl)

NameError: name 'training_loss' is not defined